In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import time
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
data = pd.read_csv('/Users/ankusmanish/Desktop/Chatbot/trained_questions - trained_questions.csv')

In [3]:
data

,question,answer
0,How long is the fellowship program?,The program is 4 months on a full-time basis.
1,How much does fellowship program cost?,The program is free to the fellows. You do not...
2,What is the fellowship program?,Coding jobs with emerging tech product compani...
3,Can the fellowship program be done remotely?,No! We believe that interaction with the mento...
4,How do I get in?,You will require to register for one of our re...
5,How do I register for the fellowship program?,You can register here: http://codingtest.bridg...
6,Do fellowship program offer any stipend or liv...,We do not offer any stipend
7,Do fellowship program offer any package or liv...,We help connect you to the right PG partners w...
8,What do I need to know to be able to get throu...,Try our sample MCQ questions from the Coding C...
9,What is the minimum qualification for the fell...,All the candidates need to be Engineers or MCA...


In [4]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

In [5]:
questions = []

question = data['question'].values

for i in range(len(data['question'])):
    text = question[i].lower()
    text = re.sub('[?]', '', text)
    text = [stemmer.stem(word) for word in word_tokenize(text) if word not in stop_words]
    text = ' '.join(text)
    questions.append(text)

In [6]:
answers = []

answer = data['answer'].values

for i in range(len(data['answer'])):
    text = answer[i].lower()
    text = re.sub('[?]', '', text)
    text = [stemmer.stem(word) for word in word_tokenize(text) if word not in stop_words]
    text = ' '.join(text)
    answers.append(text)

In [8]:
answers = list()
for i in range( len( data['answer'] ) ) :
    answers.append( '<START> ' + data['answer'][i] + ' <END>' )


In [10]:
tokenizer = Tokenizer(oov_token = '<UNK>')
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index ) + 1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 369


In [11]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )


(35, 7) 7
(35, 78) 78
(35, 78, 369)


In [56]:

encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model_26"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_30 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 200)    73800       input_29[0][0]                   
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 200)    73800       input_30[0][0]                   
___________________________________________________________________________________________

In [57]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=400 ) 

Epoch 1/400
35/35 [==============================] - 3s 74ms/sample - loss: 2.6458
Epoch 2/400
35/35 [==============================] - 0s 13ms/sample - loss: 2.6382
Epoch 3/400
35/35 [==============================] - 0s 12ms/sample - loss: 2.6281
Epoch 4/400
35/35 [==============================] - 0s 12ms/sample - loss: 2.5466
Epoch 5/400
35/35 [==============================] - 0s 12ms/sample - loss: 2.3990
Epoch 6/400
35/35 [==============================] - 0s 12ms/sample - loss: 2.3465
Epoch 7/400
35/35 [==============================] - 0s 12ms/sample - loss: 2.3253
Epoch 8/400
35/35 [==============================] - 0s 12ms/sample - loss: 2.3103
Epoch 9/400
35/35 [==============================] - 0s 12ms/sample - loss: 2.2998
Epoch 10/400
35/35 [==============================] - 0s 12ms/sample - loss: 2.2955
Epoch 11/400
35/35 [==============================] - 0s 12ms/sample - loss: 2.2863
Epoch 12/400
35/35 [==============================] - 0s 12ms/sample - loss: 2.2853
E

Epoch 99/400
35/35 [==============================] - 0s 12ms/sample - loss: 1.1185
Epoch 100/400
35/35 [==============================] - 0s 13ms/sample - loss: 1.1185
Epoch 101/400
35/35 [==============================] - 0s 12ms/sample - loss: 1.1118
Epoch 102/400
35/35 [==============================] - 0s 12ms/sample - loss: 1.1230
Epoch 103/400
35/35 [==============================] - 0s 12ms/sample - loss: 1.0711
Epoch 104/400
35/35 [==============================] - 0s 13ms/sample - loss: 1.0620
Epoch 105/400
35/35 [==============================] - 1s 15ms/sample - loss: 1.0795
Epoch 106/400
35/35 [==============================] - 1s 18ms/sample - loss: 1.1461
Epoch 107/400
35/35 [==============================] - 1s 17ms/sample - loss: 1.0491
Epoch 108/400
35/35 [==============================] - 1s 15ms/sample - loss: 1.0261
Epoch 109/400
35/35 [==============================] - 1s 22ms/sample - loss: 0.9992
Epoch 110/400
35/35 [==============================] - 1s 16ms/sam

Epoch 196/400
35/35 [==============================] - 0s 12ms/sample - loss: 0.2566
Epoch 197/400
35/35 [==============================] - 0s 12ms/sample - loss: 0.2591
Epoch 198/400
35/35 [==============================] - 0s 12ms/sample - loss: 0.2360
Epoch 199/400
35/35 [==============================] - 0s 13ms/sample - loss: 0.2301
Epoch 200/400
35/35 [==============================] - 0s 14ms/sample - loss: 0.2225
Epoch 201/400
35/35 [==============================] - 1s 14ms/sample - loss: 0.2231
Epoch 202/400
35/35 [==============================] - 0s 12ms/sample - loss: 0.2122
Epoch 203/400
35/35 [==============================] - 0s 13ms/sample - loss: 0.2078
Epoch 204/400
35/35 [==============================] - 0s 12ms/sample - loss: 0.1987
Epoch 205/400
35/35 [==============================] - 0s 12ms/sample - loss: 0.1968
Epoch 206/400
35/35 [==============================] - 0s 13ms/sample - loss: 0.1911
Epoch 207/400
35/35 [==============================] - 0s 12ms/sa

Epoch 293/400
35/35 [==============================] - 0s 12ms/sample - loss: 0.0149
Epoch 294/400
35/35 [==============================] - 0s 12ms/sample - loss: 0.0147
Epoch 295/400
35/35 [==============================] - 0s 11ms/sample - loss: 0.0142
Epoch 296/400
35/35 [==============================] - 0s 12ms/sample - loss: 0.0131
Epoch 297/400
35/35 [==============================] - 0s 12ms/sample - loss: 0.0124
Epoch 298/400
35/35 [==============================] - 1s 15ms/sample - loss: 0.0118
Epoch 299/400
35/35 [==============================] - 0s 14ms/sample - loss: 0.0114
Epoch 300/400
35/35 [==============================] - 1s 15ms/sample - loss: 0.0110
Epoch 301/400
35/35 [==============================] - 1s 15ms/sample - loss: 0.0108
Epoch 302/400
35/35 [==============================] - 1s 15ms/sample - loss: 0.0104
Epoch 303/400
35/35 [==============================] - 1s 17ms/sample - loss: 0.0103
Epoch 304/400
35/35 [==============================] - 1s 14ms/sa

Epoch 389/400
35/35 [==============================] - 0s 12ms/sample - loss: 0.0010
Epoch 390/400
35/35 [==============================] - 0s 12ms/sample - loss: 7.7490e-04
Epoch 391/400
35/35 [==============================] - 0s 12ms/sample - loss: 7.2492e-04
Epoch 392/400
35/35 [==============================] - 0s 11ms/sample - loss: 6.9092e-04
Epoch 393/400
35/35 [==============================] - 0s 13ms/sample - loss: 6.6435e-04
Epoch 394/400
35/35 [==============================] - 0s 14ms/sample - loss: 6.4159e-04
Epoch 395/400
35/35 [==============================] - 0s 13ms/sample - loss: 6.2118e-04
Epoch 396/400
35/35 [==============================] - 0s 12ms/sample - loss: 6.0244e-04
Epoch 397/400
35/35 [==============================] - 0s 12ms/sample - loss: 5.8496e-04
Epoch 398/400
35/35 [==============================] - 0s 12ms/sample - loss: 5.6845e-04
Epoch 399/400
35/35 [==============================] - 0s 12ms/sample - loss: 5.5273e-04
Epoch 400/400
35/35 [====

In [58]:

def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


In [59]:

def str_to_tokens( sentence):
    words = sentence.lower().split()
    tokens_list = tokenizer.texts_to_sequences([words])
    return pad_sequences( tokens_list , maxlen=maxlen_questions , padding='post')


In [ ]:

enc_model , dec_model = make_inference_models()

for _ in range(10):
    input_val =  input( 'Enter question : ' ) 
    states_values = [stemmer.stem(word) for word in word_tokenize(input_val) if word not in stop_words]
    states_values = ' '.join(states_values)
    states_values = enc_model.predict( str_to_tokens( states_values ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 
    
    decoded_translation_split = [word for word in decoded_translation.split()]
    for i in decoded_translation_split:
        if i == 'end':
            decoded_translation_split.remove('end')
            decoded_translation = ' '.join(decoded_translation_split)
    print( decoded_translation )


Enter question : how can i register for brigelabz
you will require to register for one of our recruitment drives and pass our hackathon challenge which includes a technical interview
Enter question : where is bridgelabz
bridgelabz solutions llp no 42 14th main 15th cross sector 4 opp to bda complex near kumarakom restaurant hsr layout bangalore 560034
Enter question : where is the training
you will require to register for one of our recruitment drives and pass our hackathon challenge which includes a technical interview
Enter question : where is the training held
mumbai and bangalore depending on your convenience
Enter question : the cost of fellowship program
the program is free to the fellows you do not have to pay anything for the program
Enter question : what are the tools i will be working on
the first interview depending on your skills demonstrated in your application and challenge submission will involve some or all of the following motivation and why you are interested in the f